In [4]:
import os

import sys
os.chdir("/Users/zyxu/Documents/py/kris")
import numpy as np
import pandas as pd
import h5py
import hdf5plugin

import func.prepro as pp
import gc

In [5]:
DATA_DIR = "/open-problems-multimodal"
FP_CELL_METADATA = os.path.join(DATA_DIR,"metadata.csv")

FP_CITE_TRAIN_INPUTS = os.path.join(DATA_DIR,"train_cite_inputs.h5")
FP_CITE_TRAIN_TARGETS = os.path.join(DATA_DIR,"train_cite_targets.h5")
FP_CITE_TEST_INPUTS = os.path.join(DATA_DIR,"test_cite_inputs.h5")

FP_MULTIOME_TRAIN_INPUTS = os.path.join(DATA_DIR,"train_multi_inputs.h5")
FP_MULTIOME_TRAIN_TARGETS = os.path.join(DATA_DIR,"train_multi_targets.h5")
FP_MULTIOME_TEST_INPUTS = os.path.join(DATA_DIR,"test_multi_inputs.h5")

FP_SUBMISSION = os.path.join(DATA_DIR,"sample_submission.csv")
FP_EVALUATION_IDS = os.path.join(DATA_DIR,"evaluation_ids.csv")

In [23]:
def readH5pyFile(filename):
    with h5py.File(filename, "r") as f:
        # Print all root level object names (aka keys) 
        # these can be group or dataset names 
        print(f"Keys in {filename}: {f.keys()}")
        # get first object name/key; may or may NOT be a group
        a_group_key = list(f.keys())[0]
        # print(a_group_key)

        # get the object type for a_group_key: usually group or dataset
        # print(type(f[a_group_key])) 

        # If a_group_key is a group name, 
        # this gets the object names in the group and returns as a list
        data = list(f[a_group_key])
        
        group = f[a_group_key]      # returns as a h5py dataset object

        # print(f"keys in lower level {list(group.keys())}")

        d = {}
        for i in list(group.keys()):
            d[i] = group[i][()]
        
        # for key, val in d.items():
        #     print(f"{key} shape is {val.shape}")
    return d

In [24]:
# train_multi_inputs = readH5pyFile(FP_MULTIOME_TRAIN_INPUTS)
# train_multi_targets = readH5pyFile(FP_MULTIOME_TRAIN_TARGETS)
# test_multi_inputs = readH5pyFile(FP_MULTIOME_TEST_INPUTS)

train_cite_inputs = readH5pyFile(FP_CITE_TRAIN_INPUTS)
train_cite_targets = readH5pyFile(FP_CITE_TRAIN_TARGETS)
test_cite_inputs = readH5pyFile(FP_CITE_TEST_INPUTS)

Keys in test_cite_inputs.h5: <KeysViewHDF5 ['test_cite_inputs']>
keys in lower level ['axis0', 'axis1', 'block0_items', 'block0_values']


In [13]:
np.char.decode(train_cite_inputs['axis0'])
np.char.decode(train_cite_inputs['axis1'])

array(['45006fe3e4c8', 'd02759a80ba2', 'c016c6b0efa5', ...,
       'a91f1b55a520', '3a9882c98205', 'c91b6b2ccd3d'], dtype='<U12')

In [16]:
np.char.decode(train_cite_targets['axis0'])
np.char.decode(train_cite_targets['axis1'])

array(['45006fe3e4c8', 'd02759a80ba2', 'c016c6b0efa5', ...,
       'a91f1b55a520', '3a9882c98205', 'c91b6b2ccd3d'], dtype='<U12')

In [17]:
np.char.decode(test_cite_inputs['axis0'])
np.char.decode(test_cite_inputs['axis1'])

array(['c2150f55becb', '65b7edf8a4da', 'c1b26cb1057b', ...,
       'a3cbc5aa0ec3', '75b350243add', 'ad5a949989b2'], dtype='<U12')

In [20]:
pd.DataFrame(train_cite_targets['block0_values'])

,0,1,2,3,4,5,6,7,8,9,...,130,131,132,133,134,135,136,137,138,139
0,1.167804,0.622530,0.106959,0.324989,3.331674,6.426002,1.480766,-0.728392,-0.468851,-0.073285,...,-0.448390,3.220174,-0.533004,0.674956,-0.006187,0.682148,1.398105,0.414292,1.780314,0.548070
1,0.818970,0.506009,1.078682,6.848758,3.524885,5.279456,4.930438,2.069372,0.333652,-0.468088,...,0.323613,8.407108,0.131301,0.047607,-0.243628,0.547864,1.832587,0.982308,2.736507,2.184063
2,-0.356703,-0.422261,-0.824493,1.137495,0.518924,7.221962,-0.375034,1.738071,0.142919,-0.971460,...,1.348692,4.888579,-0.279483,-0.131097,-0.177604,-0.689188,9.013709,-1.182975,3.958148,2.868600
3,-1.201507,0.149115,2.022468,6.021595,7.258670,2.792436,21.708519,-0.137913,1.649969,-0.754680,...,1.504426,12.391979,0.511394,0.587863,-0.752638,1.714851,3.893782,1.799661,1.537249,4.407671
4,-0.100404,0.697461,0.625836,-0.298404,1.369898,3.254521,-1.659380,0.643531,0.902710,1.291877,...,0.777023,6.496499,0.279898,-0.841950,-0.869419,0.675092,5.259685,-0.835379,9.631781,1.765445
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
70983,0.905420,0.386141,0.961590,5.090580,2.854346,6.093729,-0.586178,0.452389,0.040806,0.191407,...,1.261118,3.092832,0.003275,0.278930,-0.272002,0.249477,3.789460,0.138330,1.466193,4.278504
70984,2.101247,2.117462,0.112699,2.065512,2.176803,3.900090,-0.586001,-0.175479,1.363232,0.109905,...,0.714624,5.029233,0.909861,0.057322,2.633387,1.340077,11.456146,-1.431453,5.275882,2.510530
70985,1.221313,0.476566,1.437551,5.135631,2.926102,1.615081,-0.586910,1.760421,1.944711,-0.095096,...,-0.176027,5.027534,-0.703609,1.139491,-0.078092,1.592960,9.358179,0.981883,6.911032,3.415310
70986,-0.151433,-0.850024,0.461556,3.546561,1.996473,5.702821,0.883038,1.309014,1.029737,-0.072851,...,-0.484493,12.883892,1.579381,-0.382835,-0.065286,-0.021458,7.372662,1.010247,1.864805,3.449289
